In [ ]:
import bs4
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url="https://www.airlinequality.com/airline-reviews/british-airways"
response=requests.get(url)
response.status_code

200

In [ ]:
soup=BeautifulSoup(response.text,'html5lib')
print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7 lt-ie10" lang="en-GB"> <![endif]-->
<!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8 lt-ie10" lang="en-GB"> <![endif]-->
<!--[if IE 8]>    <html class="no-js lt-ie9 lt-ie10" lang="en-GB"> <![endif]-->
<!--[if IE 9]>    <html class="no-js lt-ie10" lang="en-GB"> <![endif]-->
<!--[if gt IE 8]><!-->
<html lang="en-GB">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <title>
   British Airways Customer Reviews - SKYTRAX
  </title>
  <!-- Google Chrome Frame for IE -->
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <!-- mobile meta -->
  <meta content="True" name="HandheldFriendly"/>
  <meta content="320" name="MobileOptimized"/>
  <meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
  <!-- icons & favicons -->
  <link href="https://www.airlinequality.com/wp-content/themes/airlinequality2014new/library/images/apple-i

## Scraping all pages

In [ ]:
import time
import requests
import re
from bs4 import BeautifulSoup

def ref_rev(base_url,AR):
    dic={'AI':131,'BA':378,'ET':176,'EM':234,'QA':239}

    reviews = []
    dates = []
    country=[]

    for i in range(1, dic.get(AR) + 1):
        url = f"{base_url}/page/{i}"
        response = requests.get(url)
        data = response.text
        soup = BeautifulSoup(data, 'html5lib')

        h2_elements = soup.body.find_all('h2', class_="text_header")
        h3_elements = soup.find_all('time')
        h3_country = soup.find_all(class_="text_sub_header userStatusWrapper")
        for tag in h3_country:
            text = tag.get_text()
            country_match = re.search(r'\((.*?)\)', text)
            if country_match:
                country.append(country_match.group(1))



        h2_texts = [h2.text for h2 in h2_elements[0:10]]
        h3_texts = [h3.text for h3 in h3_elements[0:10]]

        reviews.extend(h2_texts)
        dates.extend(h3_texts)


    return dates, reviews,country




In [ ]:
dates0,reviews0,country0= ref_rev("https://www.airlinequality.com/airline-reviews/british-airways",'BA')
dates1,reviews1,country1=ref_rev("https://www.airlinequality.com/airline-reviews/air-india",'AI')
dates2,reviews2,country2=ref_rev("https://www.airlinequality.com/airline-reviews/emirates",'EM')
dates3,reviews3,country3=ref_rev("https://www.airlinequality.com/airline-reviews/etihad-airways",'ET')
dates4,reviews4,country4=ref_rev("https://www.airlinequality.com/airline-reviews/qatar-airways",'QA')

### **Creating Dataframes for each Airline**

In [ ]:
Set_={'Date':dates0[:-1],'Customer Review':reviews0[:-1],'Country':country0}
BA_reviews=pd.DataFrame(Set_)

Set_1={'Date':dates1,'Customer Review':reviews1,'Country':country1}
Air_Ind_reviews=pd.DataFrame(Set_1)

Set_2={'Date':dates2[:-1],'Customer Review':reviews2[:-1],'Country':country2}
Emirates_reviews=pd.DataFrame(Set_2)

Set_3={'Date':dates3,'Customer Review':reviews3,'Country':country3}
Ethihad_reviews=pd.DataFrame(Set_3)

Set_4={'Date':dates4,'Customer Review':reviews4,'Country':country4}
Qatar_reviews=pd.DataFrame(Set_4)



### **Dropping all the Incorrect Data**



In [ ]:
Air_Ind_reviews=Air_Ind_reviews[~Air_Ind_reviews.apply(lambda row: row.astype(str).str.contains('Air India customer review').any(), axis=1)]
BA_reviews=BA_reviews[~BA_reviews.apply(lambda row: row.astype(str).str.contains('British Airways customer review').any(), axis=1)]
Emirates_reviews=Emirates_reviews[~Emirates_reviews.apply(lambda row: row.astype(str).str.contains('Emirates customer review').any(), axis=1)]
Ethihad_reviews=Ethihad_reviews[~Ethihad_reviews.apply(lambda row: row.astype(str).str.contains('Ethihad Airways customer review').any(), axis=1)]
Qatar_reviews=Qatar_reviews[~Qatar_reviews.apply(lambda row: row.astype(str).str.contains('Qatar Airways customer review').any(), axis=1)]


### **Fixing the Date Format**

In [ ]:
def date_1(dat):
    dat["Dates"]=pd.to_datetime(dat["Date"],format='mixed')
    dat["Date"]=dat["Dates"]
    dat.drop(columns=["Dates"],inplace=True)
    return dat

data=[Air_Ind_reviews,BA_reviews,Emirates_reviews,Ethihad_reviews,Qatar_reviews]
for df in data:
    df=date_1(df)

<ipython-input-66-035611235359>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat["Dates"]=pd.to_datetime(dat["Date"],format='mixed')
<ipython-input-66-035611235359>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat["Date"]=dat["Dates"]
<ipython-input-66-035611235359>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d

In [ ]:
Qatar_reviews

,Date,Customer Review,Country
0,2024-04-08,"""an enjoyable experience""",Kuwait
1,2024-04-05,"""compensate us for half of the bags value""",Netherlands
2,2024-04-03,"""not enough food for the 6h flight""",Germany
3,2024-04-03,"""a great flight""",United Kingdom
4,2024-04-03,"""What a disappointment""",United Kingdom
...,...,...,...
1871,2015-09-04,"""Boeing 777 cabin very spacious""",Australia
1872,2015-09-03,"""great service and attention""",Spain
1873,2015-09-01,"""flights full of children""",United Kingdom
1874,2015-09-01,"""overall a very pleasant experience""",Denmark


In [ ]:
Qatar_reviews["Customer Review"].value_counts().sort_values(ascending=False)

Customer Review
"one of the best airlines"                         5
"an amazing experience"                            3
"a pleasant experience"                            3
"Excellent service provided"                       3
"a great experience"                               3
                                                  ..
"a good flight let down by poor food"              1
"everything functioned to perfection this time"    1
"seats were comfortable and spacious"              1
"the seats were comfortable"                       1
"onboard service was good"                         1
Name: count, Length: 1810, dtype: int64

# NLP

In [ ]:
import re
def clean(text):
    text = re.sub('[^A-Za-z]+', ' ', str(text))
    return text

# Cleaning the text in the review column
data=[Air_Ind_reviews,BA_reviews,Emirates_reviews,Ethihad_reviews,Qatar_reviews]
for df in data:
    df['Cleaned Reviews']=df['Customer Review'].apply(clean)


<ipython-input-72-54542da1582f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cleaned Reviews']=df['Customer Review'].apply(clean)
<ipython-input-72-54542da1582f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cleaned Reviews']=df['Customer Review'].apply(clean)


In [ ]:
Qatar_reviews

,Date,Customer Review,Country,Cleaned Reviews
0,2024-04-08,"""an enjoyable experience""",Kuwait,an enjoyable experience
1,2024-04-05,"""compensate us for half of the bags value""",Netherlands,compensate us for half of the bags value
2,2024-04-03,"""not enough food for the 6h flight""",Germany,not enough food for the h flight
3,2024-04-03,"""a great flight""",United Kingdom,a great flight
4,2024-04-03,"""What a disappointment""",United Kingdom,What a disappointment
...,...,...,...,...
1871,2015-09-04,"""Boeing 777 cabin very spacious""",Australia,Boeing cabin very spacious
1872,2015-09-03,"""great service and attention""",Spain,great service and attention
1873,2015-09-01,"""flights full of children""",United Kingdom,flights full of children
1874,2015-09-01,"""overall a very pleasant experience""",Denmark,overall a very pleasant experience


In [ ]:
import nltk

"""This punkt tokenizer divides a text into a list of sentences by using an unsupervised algorithm to build a model for abbreviation words,
collocations, and words that start sentences. """

nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.corpus import wordnet

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
#The nltk.corpus package defines a collection of corpus reader classes, which can be used to access the contents of a diverse set of corpora.
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

# POS tagger dictionary
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('english')):
          newlist.append(tuple([word, pos_dict.get(tag[0])]))
    return newlist

for df in data:
    df['POS tagged']=df['Cleaned Reviews'].apply(token_stop_pos)

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
<ipython-input-75-98f1ac765001>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['POS tagged']=df['Cleaned Reviews'].apply(token_stop_pos)
<ipython-input-75-98f1ac765001>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['POS tagged']=df['Cleaned Reviews'].apply(token_stop_pos)


In [ ]:
Qatar_reviews.head(6)

,Date,Customer Review,Country,Cleaned Reviews,POS tagged
0,2024-04-08,"""an enjoyable experience""",Kuwait,an enjoyable experience,"[(enjoyable, a), (experience, n)]"
1,2024-04-05,"""compensate us for half of the bags value""",Netherlands,compensate us for half of the bags value,"[(compensate, v), (us, None), (half, n), (bags..."
2,2024-04-03,"""not enough food for the 6h flight""",Germany,not enough food for the h flight,"[(enough, r), (food, n), (h, n), (flight, n)]"
3,2024-04-03,"""a great flight""",United Kingdom,a great flight,"[(great, a), (flight, n)]"
4,2024-04-03,"""What a disappointment""",United Kingdom,What a disappointment,"[(disappointment, n)]"
5,2024-04-01,"""happy to fly with Qatar again""",United Kingdom,happy to fly with Qatar again,"[(happy, a), (fly, v), (Qatar, n)]"


In [ ]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
     if not pos:
        lemma = word
        lemma_rew = lemma_rew + " " + lemma
     else:
        lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
        lemma_rew = lemma_rew + " " + lemma
    return lemma_rew

for df in data:
    df['Lemma']=df['POS tagged'].apply(lemmatize)
Emirates_reviews.head()

<ipython-input-77-6faf05c70048>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Lemma']=df['POS tagged'].apply(lemmatize)
<ipython-input-77-6faf05c70048>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Lemma']=df['POS tagged'].apply(lemmatize)


,Date,Customer Review,Country,Cleaned Reviews,POS tagged,Lemma
0,2024-03-20,"""absolutely unacceptable""",Thailand,absolutely unacceptable,"[(absolutely, r), (unacceptable, a)]",absolutely unacceptable
1,2024-03-17,"""Very disappointed""",United Arab Emirates,Very disappointed,"[(disappointed, a)]",disappointed
2,2024-03-13,"""will never recommend Emirates""",Pakistan,will never recommend Emirates,"[(never, r), (recommend, v), (Emirates, n)]",never recommend Emirates
3,2024-03-10,"""Don’t book this airline!""",Germany,Don t book this airline,"[(book, n), (airline, n)]",book airline
4,2024-03-09,"""plenty of drinks available""",Australia,plenty of drinks available,"[(plenty, n), (drinks, n), (available, a)]",plenty drink available


## Analyze the Sentiment

In [ ]:
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.1 MB/s eta 0:00:00


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# function to calculate vader sentiment
def vadersentimentanalysis(review):
    vs = analyzer.polarity_scores(review)
    return vs['compound']
for df in data:
    df['Sentiment'] = df['Lemma'].apply(vadersentimentanalysis)

# function to analyse
def vader_analysis(compound):
    if compound >= 0.4:
        return 'Positive'
    elif compound < 0 :
        return 'Negative'
    else:
        return 'Neutral'
for df in data:
    df['Analysis'] = df['Sentiment'].apply(vader_analysis)


<ipython-input-79-317db0ca5d41>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sentiment'] = df['Lemma'].apply(vadersentimentanalysis)
<ipython-input-79-317db0ca5d41>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sentiment'] = df['Lemma'].apply(vadersentimentanalysis)
<ipython-input-79-317db0ca5d41>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [ ]:
Qatar_reviews

,Date,Customer Review,Country,Cleaned Reviews,POS tagged,Lemma,Sentiment,Analysis
0,2024-04-08,"""an enjoyable experience""",Kuwait,an enjoyable experience,"[(enjoyable, a), (experience, n)]",enjoyable experience,0.4404,Positive
1,2024-04-05,"""compensate us for half of the bags value""",Netherlands,compensate us for half of the bags value,"[(compensate, v), (us, None), (half, n), (bags...",compensate us half bag value,0.3400,Neutral
2,2024-04-03,"""not enough food for the 6h flight""",Germany,not enough food for the h flight,"[(enough, r), (food, n), (h, n), (flight, n)]",enough food h flight,0.0000,Neutral
3,2024-04-03,"""a great flight""",United Kingdom,a great flight,"[(great, a), (flight, n)]",great flight,0.6249,Positive
4,2024-04-03,"""What a disappointment""",United Kingdom,What a disappointment,"[(disappointment, n)]",disappointment,-0.5106,Negative
...,...,...,...,...,...,...,...,...
1871,2015-09-04,"""Boeing 777 cabin very spacious""",Australia,Boeing cabin very spacious,"[(Boeing, n), (cabin, a), (spacious, a)]",Boeing cabin spacious,0.0000,Neutral
1872,2015-09-03,"""great service and attention""",Spain,great service and attention,"[(great, a), (service, n), (attention, n)]",great service attention,0.6249,Positive
1873,2015-09-01,"""flights full of children""",United Kingdom,flights full of children,"[(flights, n), (full, a), (children, n)]",flight full child,0.0000,Neutral
1874,2015-09-01,"""overall a very pleasant experience""",Denmark,overall a very pleasant experience,"[(overall, a), (pleasant, a), (experience, n)]",overall pleasant experience,0.5106,Positive


In [ ]:
def rat(col):
    if(col=="Positive"):
        score=5
    elif (col=="Neutral"):
        score=2.5
    else:
        score=1
    return score

for df in data:
    df["Rating"]=df["Analysis"].apply(rat)

Qatar_reviews

<ipython-input-81-ea77932e8262>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Rating"]=df["Analysis"].apply(rat)
<ipython-input-81-ea77932e8262>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Rating"]=df["Analysis"].apply(rat)


,Date,Customer Review,Country,Cleaned Reviews,POS tagged,Lemma,Sentiment,Analysis,Rating
0,2024-04-08,"""an enjoyable experience""",Kuwait,an enjoyable experience,"[(enjoyable, a), (experience, n)]",enjoyable experience,0.4404,Positive,5.0
1,2024-04-05,"""compensate us for half of the bags value""",Netherlands,compensate us for half of the bags value,"[(compensate, v), (us, None), (half, n), (bags...",compensate us half bag value,0.3400,Neutral,2.5
2,2024-04-03,"""not enough food for the 6h flight""",Germany,not enough food for the h flight,"[(enough, r), (food, n), (h, n), (flight, n)]",enough food h flight,0.0000,Neutral,2.5
3,2024-04-03,"""a great flight""",United Kingdom,a great flight,"[(great, a), (flight, n)]",great flight,0.6249,Positive,5.0
4,2024-04-03,"""What a disappointment""",United Kingdom,What a disappointment,"[(disappointment, n)]",disappointment,-0.5106,Negative,1.0
...,...,...,...,...,...,...,...,...,...
1871,2015-09-04,"""Boeing 777 cabin very spacious""",Australia,Boeing cabin very spacious,"[(Boeing, n), (cabin, a), (spacious, a)]",Boeing cabin spacious,0.0000,Neutral,2.5
1872,2015-09-03,"""great service and attention""",Spain,great service and attention,"[(great, a), (service, n), (attention, n)]",great service attention,0.6249,Positive,5.0
1873,2015-09-01,"""flights full of children""",United Kingdom,flights full of children,"[(flights, n), (full, a), (children, n)]",flight full child,0.0000,Neutral,2.5
1874,2015-09-01,"""overall a very pleasant experience""",Denmark,overall a very pleasant experience,"[(overall, a), (pleasant, a), (experience, n)]",overall pleasant experience,0.5106,Positive,5.0


In [ ]:
Qatar_reviews["Analysis"].value_counts()

Analysis
Positive    956
Neutral     664
Negative    242
Name: count, dtype: int64

In [ ]:
cols=["Date","Country","Customer Review", "Analysis", "Rating"]
Air_india_Rating=Air_Ind_reviews[cols]
BA_Rating=BA_reviews[cols]
Emirates_Rating=Emirates_reviews[cols]
Ethihad_Rating=Ethihad_reviews[cols]
Qatar_Rating=Qatar_reviews[cols]

In [ ]:
Qatar_Rating

,Date,Country,Customer Review,Analysis,Rating
0,2024-04-08,Kuwait,"""an enjoyable experience""",Positive,5.0
1,2024-04-05,Netherlands,"""compensate us for half of the bags value""",Neutral,2.5
2,2024-04-03,Germany,"""not enough food for the 6h flight""",Neutral,2.5
3,2024-04-03,United Kingdom,"""a great flight""",Positive,5.0
4,2024-04-03,United Kingdom,"""What a disappointment""",Negative,1.0
...,...,...,...,...,...
1871,2015-09-04,Australia,"""Boeing 777 cabin very spacious""",Neutral,2.5
1872,2015-09-03,Spain,"""great service and attention""",Positive,5.0
1873,2015-09-01,United Kingdom,"""flights full of children""",Neutral,2.5
1874,2015-09-01,Denmark,"""overall a very pleasant experience""",Positive,5.0


### Data Warehousing

In [ ]:
Air_india_Rating.to_csv("/content/Air_Ind_Ratings.csv", index=False)
BA_Rating.to_csv("/content/BA_Ratings.csv", index=False)
Emirates_Rating.to_csv("/content/Emirates_Ratings.csv", index=False)
Ethihad_Rating.to_csv("/content/Ethihad_Ratings.csv", index=False)
Qatar_Rating.to_csv("/content/Qatar_Ratings.csv", index=False)
